In [1]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, StratifiedKFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, FunctionTransformer

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

import warnings
warnings.filterwarnings('ignore')

In [2]:
seed = 42
splits = 5
repeats = 4
kf = KFold(n_splits=splits)
skf = StratifiedKFold(n_splits=splits)
rskf = RepeatedStratifiedKFold(n_splits=splits, random_state=seed, n_repeats=repeats)
np.random.seed(seed)

In [38]:
train = pd.read_csv("data/Train.csv")
test = pd.read_csv("data/Test.csv")
submission = pd.read_csv("data/SampleSubmission.csv")

df = train.copy()

# Pre-Processing

In [4]:
def drop_cols(df):
    # Dropping columns with more than 90% missing values
    missing_percent = df.isnull().mean()
    columns_to_drop = missing_percent[missing_percent > 0.9].index
    df.drop(columns=columns_to_drop, inplace=True)
    
    # Dropping id columns
    df.drop('id', axis=1, inplace=True)
    df.drop('site_id', axis=1, inplace=True)
    df.drop('date', axis=1, inplace=True)

    # City and Country can be dropped since we have site_latitude and site_longitude
    df.drop('city', axis=1, inplace=True)
    df.drop('country', axis=1, inplace=True)

    return df 

In [39]:
df = drop_cols(df)

In [6]:
knn_imputer = make_pipeline(KNNImputer(n_neighbors=5, weights="distance"))
imputer = make_pipeline(SimpleImputer(strategy="mean"))

In [7]:
cols_with_missing_values = df.columns[df.isnull().any()].tolist()

In [8]:
std = make_pipeline(StandardScaler())
min_max = make_pipeline(MinMaxScaler())
rob = make_pipeline(RobustScaler())

In [9]:
numeric_features = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

In [10]:
transformer = make_column_transformer(
    (knn_imputer, cols_with_missing_values),
    remainder='passthrough'
)

In [11]:
preprocessor = make_pipeline(transformer, std)

In [40]:
X = df.drop("pm2_5", axis=1)
y = df.pm2_5

In [41]:
preprocessed_train = preprocessor.fit_transform(X)
preprocessed_train_df = pd.DataFrame(preprocessed_train, columns=preprocessor.get_feature_names_out())
preprocessed_train_df

,pipeline__sulphurdioxide_so2_column_number_density,pipeline__sulphurdioxide_so2_column_number_density_amf,pipeline__sulphurdioxide_so2_slant_column_number_density,pipeline__sulphurdioxide_cloud_fraction,pipeline__sulphurdioxide_sensor_azimuth_angle,pipeline__sulphurdioxide_sensor_zenith_angle,pipeline__sulphurdioxide_solar_azimuth_angle,pipeline__sulphurdioxide_solar_zenith_angle,pipeline__sulphurdioxide_so2_column_number_density_15km,pipeline__carbonmonoxide_co_column_number_density,...,pipeline__cloud_cloud_optical_depth,pipeline__cloud_surface_albedo,pipeline__cloud_sensor_azimuth_angle,pipeline__cloud_sensor_zenith_angle,pipeline__cloud_solar_azimuth_angle,pipeline__cloud_solar_zenith_angle,remainder__site_latitude,remainder__site_longitude,remainder__hour,remainder__month
0,-0.775717,2.521295,-1.302391,1.593737,-1.040256,-1.671248,-0.786235,0.428577,-1.496416,0.420719,...,-0.252666,-2.157190,-0.998075,-0.183074,-0.729542,1.010934,2.737049,-2.858390,2.601709,1.133657
1,-1.107188,-0.959468,-0.910451,1.122000,0.963025,0.968996,-1.261823,-0.378030,-0.840587,1.113214,...,-0.275576,-0.898392,0.993476,-0.085299,-1.077957,-0.299072,2.737049,-2.858390,1.375955,1.426018
2,0.070079,1.664311,0.095986,0.360476,-0.999125,0.189458,-0.654638,1.026531,0.056361,0.615865,...,-0.237405,-1.387064,-0.971575,1.073604,-0.816199,1.992492,2.737049,-2.858390,2.601709,1.426018
3,0.169485,0.565277,0.198251,0.476695,-0.982690,0.833586,-0.542178,1.235947,0.162279,0.916061,...,-0.454747,-1.835724,-0.979083,0.725578,-0.598487,1.247387,2.737049,-2.858390,3.827463,1.426018
4,0.812990,0.643422,1.060143,1.153346,-0.991467,0.497846,-0.876360,1.730087,0.992670,1.460975,...,-0.284505,-2.276183,-0.987811,0.317451,-0.932272,1.739687,2.737049,-2.858390,2.601709,1.426018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8066,0.035221,-0.605972,0.051371,0.095638,1.012677,-0.781505,-1.282344,0.216425,0.044282,-0.351124,...,-0.213173,1.127404,0.986466,1.105260,-1.285329,-0.703786,-0.884001,0.699719,-1.075553,-1.497596
8067,-1.886743,-0.713398,-1.839859,0.987325,-1.003391,0.463549,-0.875799,0.520834,-1.928265,0.006405,...,-0.344922,1.010576,-1.009271,-0.343817,-0.395399,0.373908,-0.884001,0.699719,0.150201,-1.497596
8068,-0.312175,-0.561413,-0.244714,1.414376,-1.027322,-0.711762,-0.067766,0.190244,-0.343911,-0.508171,...,-0.353688,0.768932,-1.017458,-0.972529,-0.765042,0.212921,-0.884001,0.699719,0.150201,-1.497596
8069,-0.665489,0.426982,-0.794404,-0.362668,1.020362,-1.144386,-1.138484,-0.578450,-0.968689,-0.019830,...,1.878519,0.738332,0.988103,1.003843,-1.377644,-0.574755,-0.884001,0.699719,-1.075553,-1.497596


In [24]:
models = [
    ('rf', RandomForestRegressor(random_state=seed)),
    ('cat', CatBoostRegressor(random_state=seed, verbose=0)),
    ('xgb', XGBRegressor(random_state=seed, verbose=0)),
    ('lgb', LGBMRegressor(random_state=seed, verbose=-1)),
    ('dart', LGBMRegressor(random_state=seed, boosting_type='dart', verbose=-1))
]
models_scores = []

cv = kf

for (label, model) in models:
    model_name = label
    y_pred = cross_val_predict(model, preprocessed_train_df, y, cv=cv)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    models_scores.append((model_name, rmse))
    print(f"RMSE for {model_name}: {rmse}\n")

RMSE for rf: 0.9857453295420019

RMSE for cat: 0.9960062240448803

RMSE for xgb: 1.1448220041176382

RMSE for lgb: 0.9778288553186316

RMSE for dart: 0.9688258612697894



In [60]:
import xgboost as xgb
import optuna

def xgb_objective(trial, data=preprocessed_train_df, target=y):
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(train_x, label=train_y)
    dvalid = xgb.DMatrix(valid_x, label=valid_y)

    param = {
        "verbosity": 0,
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    rmse = np.sqrt(mean_squared_error(valid_y, preds))
    return rmse

In [61]:
study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(xgb_objective, n_trials=100, timeout=600)

print("Number of finished trials: ", len(study_xgb.trials))
print("Best trial:")
trial_xgb = study_xgb.best_trial

print("  Value: {}".format(trial_xgb.value))
print("  Params: ")
for key, value in trial_xgb.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-29 16:24:56,442] A new study created in memory with name: no-name-731874bd-2f53-4f2c-998d-35d0262f2869
[I 2024-04-29 16:24:56,512] Trial 0 finished with value: 22.43395309622473 and parameters: {'booster': 'gbtree', 'lambda': 0.02975240886894619, 'alpha': 0.005559220150094939, 'subsample': 0.5676291183718224, 'colsample_bytree': 0.5890998913361446, 'max_depth': 3, 'min_child_weight': 5, 'eta': 0.0004673958994872632, 'gamma': 0.002784116203251504, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 22.43395309622473.
[I 2024-04-29 16:24:56,612] Trial 1 finished with value: 24.701473948109452 and parameters: {'booster': 'gbtree', 'lambda': 0.0008214971051695889, 'alpha': 2.1004722903831775e-06, 'subsample': 0.7397470205659398, 'colsample_bytree': 0.5901113846320944, 'max_depth': 7, 'min_child_weight': 6, 'eta': 0.006629709810844131, 'gamma': 0.0004404652612093052, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 22.43395309622473.
[I 2024-04-29 16:24:56,700] Trial

Number of finished trials:  100
Best trial:
  Value: 14.350524242550977
  Params: 
    booster: dart
    lambda: 0.0012725070024235399
    alpha: 3.7429005503587056e-06
    subsample: 0.7714172533021875
    colsample_bytree: 0.712986273331513
    max_depth: 3
    min_child_weight: 2
    eta: 0.4540098414902494
    gamma: 8.823134375009324e-05
    grow_policy: lossguide
    sample_type: uniform
    normalize_type: forest
    rate_drop: 8.775017290436475e-05
    skip_drop: 1.937521651806271e-07


In [62]:
import lightgbm as lgb

def lgb_objective(trial, data=preprocessed_train_df, target=y):
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    rmse = np.sqrt(mean_squared_error(valid_y, preds))
    return rmse

In [63]:
study = optuna.create_study(direction="minimize")
study.optimize(lgb_objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-29 16:28:25,961] A new study created in memory with name: no-name-347a48cb-a02a-4217-9193-dcf32e057e3f
[I 2024-04-29 16:28:26,223] Trial 0 finished with value: 15.025140107448877 and parameters: {'lambda_l1': 0.028682558240952984, 'lambda_l2': 0.1196625072234609, 'num_leaves': 132, 'feature_fraction': 0.8101199367414998, 'bagging_fraction': 0.4556930241774612, 'bagging_freq': 1, 'min_child_samples': 41}. Best is trial 0 with value: 15.025140107448877.
[I 2024-04-29 16:28:26,357] Trial 1 finished with value: 18.414194214850973 and parameters: {'lambda_l1': 1.245332952489028e-08, 'lambda_l2': 0.03955415348607147, 'num_leaves': 128, 'feature_fraction': 0.5298350000965911, 'bagging_fraction': 0.4861567236483171, 'bagging_freq': 5, 'min_child_samples': 89}. Best is trial 0 with value: 15.025140107448877.
[I 2024-04-29 16:28:26,626] Trial 2 finished with value: 16.009056697231053 and parameters: {'lambda_l1': 1.2681936261587076e-07, 'lambda_l2': 1.5600373147252863e-08, 'num_leaves

Number of finished trials: 100
Best trial:
  Value: 10.835838223338907
  Params: 
    lambda_l1: 0.27372944014965833
    lambda_l2: 0.9999491350519373
    num_leaves: 110
    feature_fraction: 0.6799941899139543
    bagging_fraction: 0.7156338881569069
    bagging_freq: 3
    min_child_samples: 25


In [42]:
test = drop_cols(test)

In [43]:
preprocessed_test = preprocessor.transform(test)
preprocessed_test_df = pd.DataFrame(preprocessed_test, columns=preprocessor.get_feature_names_out())
preprocessed_test_df

,pipeline__sulphurdioxide_so2_column_number_density,pipeline__sulphurdioxide_so2_column_number_density_amf,pipeline__sulphurdioxide_so2_slant_column_number_density,pipeline__sulphurdioxide_cloud_fraction,pipeline__sulphurdioxide_sensor_azimuth_angle,pipeline__sulphurdioxide_sensor_zenith_angle,pipeline__sulphurdioxide_solar_azimuth_angle,pipeline__sulphurdioxide_solar_zenith_angle,pipeline__sulphurdioxide_so2_column_number_density_15km,pipeline__carbonmonoxide_co_column_number_density,...,pipeline__cloud_cloud_optical_depth,pipeline__cloud_surface_albedo,pipeline__cloud_sensor_azimuth_angle,pipeline__cloud_sensor_zenith_angle,pipeline__cloud_solar_azimuth_angle,pipeline__cloud_solar_zenith_angle,remainder__site_latitude,remainder__site_longitude,remainder__hour,remainder__month
0,-0.289826,0.551538,-0.339875,-0.478182,-1.020633,-0.709346,0.079407,-0.426057,-0.431919,0.924430,...,-0.555601,0.192676,-1.016884,-1.154882,0.023704,-0.398069,2.310007,-3.244749,2.601709,0.841295
1,0.444323,0.038806,0.532797,-0.534295,1.017386,-1.041365,-0.828925,-0.742362,0.577104,0.223445,...,0.156712,0.018541,0.988230,0.607819,-0.185631,-2.043486,2.310007,-3.244749,2.601709,0.841295
2,-0.222370,2.385150,-0.320054,0.469645,0.987589,0.155741,0.025519,-1.764172,-0.371886,0.331751,...,0.186928,-2.301836,1.004745,-0.545499,-0.050124,-1.712409,2.310007,-3.244749,2.601709,0.841295
3,0.201668,-1.461192,0.165465,-1.382091,0.979661,0.402140,-1.523450,0.763132,0.121419,2.378044,...,0.392515,1.353376,0.967082,1.127969,-0.073652,-2.393929,2.310007,-3.244749,1.375955,0.841295
4,-2.119050,-0.437397,-2.190922,-1.382091,0.952271,1.379753,-0.518589,-2.380802,-1.966653,1.273870,...,-0.013758,0.272335,0.978301,0.594513,-0.447679,-1.825810,2.310007,-3.244749,1.375955,0.841295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,0.268454,-0.223388,0.305786,0.992974,0.979749,0.806075,-1.474478,-1.049983,0.461001,-0.254451,...,-0.564012,-0.088925,0.987364,0.703713,-1.532047,-1.023495,-0.341920,0.480746,-1.075553,-1.497596
2779,-0.657615,0.936198,-0.949738,0.229428,-1.018458,-0.646707,-0.884160,1.133527,-0.618426,0.970738,...,-0.168245,-0.595008,-1.029271,-1.842229,-0.918786,-0.462918,1.513728,-1.995296,1.375955,-1.205235
2780,0.165974,-1.183972,0.147374,-0.548185,1.003276,-0.540131,-1.476605,0.145164,0.130134,3.260288,...,-0.460953,-1.641780,1.011082,-0.956018,-1.532649,0.165954,1.513728,-1.995296,1.375955,-1.205235
2781,-1.308693,-0.104787,-1.349785,0.254179,0.984110,0.213050,-1.315269,0.200350,-1.324242,3.536795,...,-0.322731,-1.283619,0.982005,0.422431,-1.509792,0.360707,1.513728,-1.995296,1.375955,-1.205235


In [64]:
lgb_params = {
    "lambda_l1": 0.27372944014965833,
    "lambda_l2": 0.9999491350519373,
    "num_leaves": 110,
    "feature_fraction": 0.6799941899139543,
    "bagging_fraction": 0.7156338881569069,
   "bagging_freq": 3,
    "min_child_samples": 25
}

In [65]:
model = LGBMRegressor(**lgb_params, verbose=-1)
model.fit(preprocessed_train_df, y)

LGBMRegressor(bagging_fraction=0.7156338881569069, bagging_freq=3,
              feature_fraction=0.6799941899139543,
              lambda_l1=0.27372944014965833, lambda_l2=0.9999491350519373,
              min_child_samples=25, num_leaves=110, verbose=-1)

In [66]:
preds = model.predict(preprocessed_test_df)

In [67]:
submission["pm2_5"] = preds
submission.to_csv("./predictions/simple_lgbm.csv", index=False)